In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [ ]:
train_files_names = data['image_names'].values.tolist()

In [ ]:
import cv2

file_name = []
x = []
for i in train_files_names:
    image = cv2.imread('images/' + i)
    x.append(image)
    file_name.append(i)

x = np.array(x)

In [ ]:
y = data['emergency_or_not'].values.tolist()

## Using CNN network

In [ ]:
import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten

Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [ ]:
from keras.utils import to_categorical

y = to_categorical(y)

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation = 'relu', input_shape = (x.shape[1:])))
model.add(MaxPool2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 186624)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               23888000  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                

In [ ]:
model.compile(optimizer = 'adam', 
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.fit(x, y, epochs = 10)


Epoch 1/10
1646/1646 [==============================] - 114s 69ms/step - loss: 82.0949 - accuracy: 0.6097
Epoch 2/10
1646/1646 [==============================] - 117s 71ms/step - loss: 0.4325 - accuracy: 0.8238
Epoch 3/10
1646/1646 [==============================] - 120s 73ms/step - loss: 0.2100 - accuracy: 0.9162
Epoch 4/10
1646/1646 [==============================] - 114s 69ms/step - loss: 0.0816 - accuracy: 0.9803
Epoch 5/10
1646/1646 [==============================] - 111s 67ms/step - loss: 0.0298 - accuracy: 0.9921
Epoch 6/10
1646/1646 [==============================] - 112s 68ms/step - loss: 0.0152 - accuracy: 0.9961
Epoch 7/10
1646/1646 [==============================] - 116s 70ms/step - loss: 0.0133 - accuracy: 0.9961
Epoch 8/10
1646/1646 [==============================] - 108s 65ms/step - loss: 0.0344 - accuracy: 0.9897
Epoch 9/10
1646/1646 [==============================] - 108s 66ms/step - loss: 0.0484 - accuracy: 0.9848
Epoch 10/10
1646/1646 [==============================

## Testing the Data

In [ ]:
test_data = pd.read_csv('test_vc2kHdQ.csv')

In [ ]:
test_data.head()

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [ ]:
test_file_names = test_data['image_names'].values.tolist()

test_file_name = []
x_test = []
for i in test_file_names:
    image = cv2.imread('images/' + i)
    x_test.append(image)
    test_file_name.append(i)

x_test = np.array(x_test)

In [ ]:
model.predict(x_test)

array([[9.9999440e-01, 6.2584877e-07],
       [7.7486038e-07, 9.9999779e-01],
       [9.6589792e-01, 1.4062822e-03],
       ...,
       [4.4703484e-06, 9.9999994e-01],
       [9.9468720e-01, 5.6788890e-04],
       [9.9999857e-01, 1.5424332e-05]], dtype=float32)

In [ ]:
labels = np.argmax(model.predict(x_test), axis=-1)

## Creating submission file

In [ ]:
submission = pd.DataFrame()
submission['image_names'] = test_file_name
submission['emergency_or_not'] = labels

In [ ]:
submission.to_csv('submission.csv', index = False)